In [2]:
!pip install llama_index

INFO: pip is looking at multiple versions of llama-cloud-services to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of llama-cloud-services to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 70.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.6/284.6 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.0/41.0 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.4/40.4 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.7/309.7 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [13]:
!pip install llama_index.llms.mistralai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 10.4 MB/s eta 0:00:00


In [15]:
!pip install llama_index.embeddings.mistralai

In [8]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
import os
os.environ["MISTRAL_API_KEY"] = "your-api-key"  # для OpenAI

In [10]:
documents = SimpleDirectoryReader("/content/dd/").load_data()  # из папки
# Поддерживает PDF, TXT, Word, HTML и др.

In [11]:
index = VectorStoreIndex.from_documents(documents)

ValueError: 
******
Could not load OpenAI embedding model. If you intended to use OpenAI, please check your OPENAI_API_KEY.
Original error:
No API key found for OpenAI.
Please set either the OPENAI_API_KEY environment variable or openai.api_key prior to initialization.
API keys can be found or created at https://platform.openai.com/account/api-keys

Consider using embed_model='local'.
Visit our documentation for more embedding options: https://docs.llamaindex.ai/en/stable/module_guides/models/embeddings.html#modules
******

In [17]:
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, StorageContext
from llama_index.llms.mistralai import MistralAI
from llama_index.embeddings.mistralai import MistralAIEmbedding

# Инициализация Mistral
llm = MistralAI(api_key="MIttQcFf85fUBuQ4CxRiUVO0MNgpWlxW", model="mistral-small")
embed_model = MistralAIEmbedding(api_key="MIttQcFf85fUBuQ4CxRiUVO0MNgpWlxW", model_name="mistral-embed")

# Чтение документов
documents = SimpleDirectoryReader("/content/dd").load_data()  # Поддерживает PDF, TXT, Word, HTML

# Создание индекса с Mistral
index = VectorStoreIndex.from_documents(
    documents,
    llm=llm,
    embed_model=embed_model
)

# Сохранение индекса
index.storage_context.persist(persist_dir="storage_mistral/")

# Загрузка индекса
storage_context = StorageContext.from_defaults(persist_dir="storage_mistral/")
loaded_index = VectorStoreIndex.from_documents(
    [],
    storage_context=storage_context,
    llm=llm,
    embed_model=embed_model
)

Loading llama_index.core.storage.kvstore.simple_kvstore from storage_mistral/docstore.json.
Loading llama_index.core.storage.kvstore.simple_kvstore from storage_mistral/index_store.json.


In [19]:
# Базовый запрос
query_engine = index.as_query_engine()
response = query_engine.query("Объясни LlamaIndex простыми словами")
print(response)

# Кастомные промпты
from llama_index.core import PromptTemplate

mistral_prompt = PromptTemplate("""<s>[INST]
Контекст: {context_str}
Вопрос: {query_str}
[/INST]""")

query_engine.update_prompts(
    {"response_synthesizer:text_qa_template": mistral_prompt}
)

ValueError: 
******
Could not load OpenAI model. If you intended to use OpenAI, please check your OPENAI_API_KEY.
Original error:
No API key found for OpenAI.
Please set either the OPENAI_API_KEY environment variable or openai.api_key prior to initialization.
API keys can be found or created at https://platform.openai.com/account/api-keys

To disable the LLM entirely, set llm=None.
******

In [21]:
# Настройка параметров Mistral
llm.temperature = 0.7  # Креативность
llm.max_tokens = 2048  # Максимальная длина ответа

# Пакетная обработка
from llama_index.core.llms import ChatMessage

messages = [
    ChatMessage(role="user", content="Объясни RAG"),
    ChatMessage(role="assistant", content="Retrieval-Augmented Generation это..."),
    ChatMessage(role="user", content="Как это реализовано в LlamaIndex?")
]
response = llm.chat(messages)

In [22]:
print(response)

assistant: Retrieval-Augmented Generation (RAG) модель в LlamaIndex реализована путем комбинирования модели генерации языка с моделью поиска.

В частности, LlamaIndex использует модель поиска, которая индексирует текстовые документы и способна быстро и точно отвечать на запросы к этим документам. Когда пользователь задает вопрос, модель поиска извлекает соответствующие документы из индекса.

Затем эти документы передаются в модель генерации языка, которая использует их в качестве контекста для генерирования ответа на запрос пользователя. Это позволяет LlamaIndex предоставлять более информативные и точные ответы, поскольку модель генерации языка может использовать знания, содержащиеся в документах, для создания своих ответов.

Эта комбинация модели поиска и модели генерации языка позволяет LlamaIndex достигать более высокого уровня точности и полезности в своих ответах, чем модель генерации языка, работающая в изоляции.


In [23]:
# Настройка чанкинга
from llama_index.core.node_parser import SentenceWindowNodeParser

node_parser = SentenceWindowNodeParser(
    window_size=3,
    window_metadata_key="window",
    original_text_metadata_key="original_text"
)

# Альтернативные ретриверы
from llama_index.core.retrievers import VectorIndexRetriever

retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=3,
    embed_model=embed_model
)

In [24]:
query_engine = index.as_query_engine(
    similarity_top_k=3,
    llm=llm
)
response = query_engine.query("Какие есть альтернативы LlamaIndex?")

In [25]:
print(response)

Based on the context provided, alternative models to LlamaIndex include Longformer, BigBird, Linformer, and LongT5. These models are designed to handle long-context tasks and may serve as alternatives to LlamaIndex in various applications. However, the specific context does not provide information about the direct comparison or evaluation of these models with LlamaIndex.


In [27]:
from llama_index.core.memory import ChatMemoryBuffer

memory = ChatMemoryBuffer.from_defaults(llm=llm)
chat_engine = index.as_chat_engine(
    chat_mode="context",
    memory=memory
)

ValueError: 
******
Could not load OpenAI model. If you intended to use OpenAI, please check your OPENAI_API_KEY.
Original error:
No API key found for OpenAI.
Please set either the OPENAI_API_KEY environment variable or openai.api_key prior to initialization.
API keys can be found or created at https://platform.openai.com/account/api-keys

To disable the LLM entirely, set llm=None.
******

In [36]:
from llama_index.core import get_response_synthesizer
from llama_index.llms.mistralai import MistralAI
from llama_index.core import PromptTemplate

# Инициализация Mistral
llm = MistralAI(
    api_key="MIttQcFf85fUBuQ4CxRiUVO0MNgpWlxW",
    model="mistral-large-latest",
    temperature=0.7
)

# Кастомный промпт для Mistral в формате INST
mistral_rag_prompt = PromptTemplate("""<s>[INST]
Ты - эксперт по анализу документов. Ответь на вопрос, используя только предоставленный контекст.
Контекст: {context_str}
Вопрос: {query_str}
Ответь подробно и точно. [/INST]""")
nodes = index.as_retriever().retrieve("Что такое RAG?")
# Инициализация синтезатора с настройками для Mistral
synth = get_response_synthesizer(
    llm=llm,
    response_mode="tree_summarize",
    text_qa_template=mistral_rag_prompt,
    streaming=False
)

# Генерация ответа
response = synth.synthesize(
    "Что такое RAG?",
    nodes = nodes
)

print(response)

The query is in Russian, asking "What is RAG?" However, there is no information provided about "RAG" in the given context. Therefore, I cannot provide an answer.


openai

In [ ]:
query_engine = index.as_query_engine()
response = query_engine.query("Что такое LlamaIndex?")
print(response)  # ответ LLM

In [ ]:
from llama_index.core import PromptTemplate

qa_prompt = PromptTemplate("""Контекст: {context_str}
Вопрос: {query_str}
Ответ:""")

query_engine.update_prompts({"response_synthesizer:text_qa_template": qa_prompt})

создание индеса

In [ ]:
documents = SimpleDirectoryReader("data/").load_data()  # из папки
# Поддерживает PDF, TXT, Word, HTML и др.
index = VectorStoreIndex.from_documents(documents)
index.storage_context.persist(persist_dir="storage/")  # сохранить
# Загрузка:
from llama_index.core import StorageContext, load_index_from_storage
storage_context = StorageContext.from_defaults(persist_dir="storage/")
loaded_index = load_index_from_storage(storage_context)

web

In [ ]:
from llama_index.readers.web import SimpleWebPageReader
urls = ["https://example.com"]
web_docs = SimpleWebPageReader().load_data(urls)

sql

In [ ]:
from llama_index.readers.database import DatabaseReader
db_reader = DatabaseReader("sqlite:///database.db")
query = "SELECT * FROM table"
db_docs = db_reader.load_data(query=query)

In [ ]:
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding

llm = OpenAI(model="gpt-4", temperature=0.7)
embed_model = OpenAIEmbedding(model="text-embedding-3-small")

In [ ]:
from llama_index.core.node_parser import SentenceSplitter
splitter = SentenceSplitter(chunk_size=512)
nodes = splitter.get_nodes_from_documents(documents)

In [ ]:
from llama_index.core.retrievers import VectorIndexRetriever, KeywordTableRetriever
vector_retriever = VectorIndexRetriever(index=index, similarity_top_k=2)
keyword_retriever = KeywordTableRetriever(index=index)

rag

In [28]:
from llama_index.core import get_response_synthesizer
synth = get_response_synthesizer(response_mode="tree_summarize")
response = synth.synthesize("Что такое RAG?", nodes)

ValueError: 
******
Could not load OpenAI model. If you intended to use OpenAI, please check your OPENAI_API_KEY.
Original error:
No API key found for OpenAI.
Please set either the OPENAI_API_KEY environment variable or openai.api_key prior to initialization.
API keys can be found or created at https://platform.openai.com/account/api-keys

To disable the LLM entirely, set llm=None.
******

agents

In [ ]:
from llama_index.core.agent import ReActAgent
agent = ReActAgent.from_tools(tools=[...], llm=llm)
response = agent.chat("Сравни LlamaIndex и LangChain")

логгирование

In [ ]:
import logging
logging.basicConfig(level=logging.INFO)

графы зависимостей

In [ ]:
from llama_index.core import KnowledgeGraphIndex
kg_index = KnowledgeGraphIndex.from_documents(documents)
kg_index.get_networkx_graph()  # для визуализации